In [1]:
import tensorflow
from PIL import Image
import glob
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, save_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
from keras.applications.imagenet_utils import preprocess_input
from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import random
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
import numpy as np
import seaborn as sns
from keras.models import Sequential 
from keras.layers import Activation, Dense, Flatten 


In [3]:

train_path='/Users/Athithyaraagul/Code/Machine_Learning/Orcal-Cancer-Pic-Data/train/'
val_path='/Users/Athithyaraagul/Code/Machine_Learning/Orcal-Cancer-Pic-Data/val/'
test_path='/Users/Athithyaraagul/Code/Machine_Learning/Orcal-Cancer-Pic-Data/test/'

IMAGE_SIZE = (224, 224, 3)

resnet = ResNet50(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
for layer in resnet.layers:
    layer.trainable = True

folders = glob.glob('/Users/Athithyaraagul/Code/Machine_Learning/Orcal-Cancer-Pic-Data/train/*')
len(folders) 

x = Flatten()(resnet.output)
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)
#model.summary()



model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model.summary()



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
plot_model(model)

In [4]:
train_datagen = ImageDataGenerator( horizontal_flip = True,
                                    vertical_flip = True)

test_datagen = ImageDataGenerator(horizontal_flip = True,
                                  vertical_flip = True)
val_datagen = ImageDataGenerator(horizontal_flip = True,
                                  vertical_flip = True)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')
testing_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (256, 256),
                                            batch_size = 63,
                                            class_mode = 'categorical',
                                              shuffle=False)
validation_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'categorical')


Found 4946 images belonging to 2 classes.
Found 126 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [5]:
callbacks = [ModelCheckpoint('.mdlwts.hdf5', monitor='val_accuracy',mode='max',verbose=1, save_best_only=True),
             ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, verbose=1, mode='max', min_lr=0.00000000001)]

In [ ]:
r = model.fit(
training_set,
validation_data=validation_set,
epochs=20,
steps_per_epoch=len(training_set),
validation_steps=len(validation_set),
callbacks=callbacks)
print(r.history.keys())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss')

In [ ]:
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
# plt.savefig('AccVal_acc')

In [ ]:
plt.plot(r.history['lr'], label='Learning Rate')
plt.legend()
plt.show()

In [ ]:
model.save("model.h5")

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/working/model.h5')
loss,acc = model.evaluate(testing_set)

In [ ]:
from tensorflow.keras.models import load_model
model=load_model('/kaggle/input/ccnn1234/Oral cancer detection.h5')
y_pred=model.predict(testing_set)
y_pred_classes=np.argmax(y_pred,axis=1)

y_true=testing_set.classes
print(y_true)

report=classification_report(y_true,y_pred_classes)
print(report)

In [ ]:
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()